In [1]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.operators.dagrun_operator import TriggerDagRunOperator
from datetime import datetime, timedelta

# Define default_args and other DAG configurations
default_args = {
    'owner': 'your_name',
    'depends_on_past': False,
    'start_date': datetime(2023, 1, 1),
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

dag = DAG(
    'jupyter_notebook_workflow',
    default_args=default_args,
    description='A DAG to run Jupyter notebooks sequentially',
    schedule_interval=None,  # Set to None for on-demand execution
)

# Function to execute Jupyter notebook
def execute_notebook(notebook_path):
    import subprocess
    subprocess.run(['jupyter', 'nbconvert', '--execute', '--to', 'notebook', '--inplace', notebook_path])

# List of notebook files
notebooks = ['test1.ipynb', 'test2.ipynb']

# Define tasks for each notebook
for notebook in notebooks:
    task_id = f'run_{notebook.replace(".ipynb", "")}'
    execute_task = PythonOperator(
        task_id=task_id,
        python_callable=execute_notebook,
        op_args=[notebook],
        dag=dag,
    )

    # Set up task dependencies
    if notebooks.index(notebook) > 0:
        execute_task.set_upstream(dag.get_task(f'run_{notebooks[notebooks.index(notebook) - 1].replace(".ipynb", "")}'))

# Define TriggerDagRunOperator to allow manual triggering of the DAG
trigger_dag = TriggerDagRunOperator(
    task_id='trigger_jupyter_notebook_workflow',
    trigger_dag_id='jupyter_notebook_workflow',
    dag=dag,
)

# Set up task dependencies to ensure TriggerDagRunOperator runs after all notebook tasks
for notebook in notebooks:
    trigger_dag.set_upstream(dag.get_task(f'run_{notebook.replace(".ipynb", "")}'))


/tmp/ipykernel_3390/2443763365.py:2 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

/tmp/ipykernel_3390/2443763365.py:3 DeprecationWarning: The `airflow.operators.dagrun_operator.TriggerDagRunOperator` class is deprecated. Please use `'airflow.operators.trigger_dagrun.TriggerDagRunOperator'`.

/tmp/ipykernel_3390/2443763365.py:17 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.